# Étude de cas - Data Science

Vous avez à votre disposition un jeu de données qui contient des identifiants anonymisés de 4 centres commerciaux français ainsi que des identifiants de téléphones dont les propriétaires ont visité ces centres commerciaux avec la date et l'heure des différents pings observés au cours de la visite.

L'objectif de l'étude est, à partir de ces données, de déterminer les horaires d'ouverture de chacun des centres pour chaque jour de la semaine.

Idéalement vous proposerez un algorithme qui permettra de déterminer les horaires d'ouverture d'un centre ne faisant pas partie de ce dataset à partir de ses données de fréquentation.

N'hésitez pas à proposer également toute piste d'amélioration sur la qualité des données source que vous jugerez pertinente.

Vous présenterez vos résultats sous la forme d'un notebook avec vos calculs éventuellement accompagné d'un powerpoint pour présenter les différents résultats.

In [133]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import re, datetime
import matplotlib.pyplot as plt

Changer le path du csv ci-dessous pour déterminer les horaires d'ouverture d'un centre ne faisant pas partie de ce dataset

In [137]:
csv = './study_centers_201909.csv'
df = pd.read_csv(csv)
df.head()

,shopping_center_id,device_local_date,device_hash_id
0,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 10:00:25.000,6fdffac307
1,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 17:13:15.000,386141ebd8
2,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 09:07:06.000,b06242b848
3,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 17:14:49.000,c13cc52e82
4,599cb959-11ef-49aa-9eb3-e6c17b4ea6ba,2019-09-14 10:17:35.000,f339ddf999


Ajout d'un index supplémentaire pour chaque shop.

Suppression des échantillons présentant de la donnée manquante.

Parser la date et l'heure.

In [135]:
df['shopping_center_index'] = pd.factorize(df.shopping_center_id)[0]
df = df.dropna()
df["device_local_date"] = pd.to_datetime(df["device_local_date"])
df['hours'] = df['device_local_date'].dt.time
df['day'] = df['device_local_date'].dt.dayofweek
df.head()

,shopping_center_id,device_local_date,device_hash_id,shopping_center_index,hours,day
0,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 10:00:25,6fdffac307,0,10:00:25,5
1,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 17:13:15,386141ebd8,0,17:13:15,5
2,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 09:07:06,b06242b848,0,09:07:06,5
3,b43e9e4f-acd1-4941-874d-e0c5650ab91e,2019-09-14 17:14:49,c13cc52e82,0,17:14:49,5
4,599cb959-11ef-49aa-9eb3-e6c17b4ea6ba,2019-09-14 10:17:35,f339ddf999,1,10:17:35,5


Initialisation du resultat final.

In [132]:
result = pd.DataFrame({'shopping_center_index': list(df['shopping_center_index'].unique()),
                    'shopping_center_id': list(df['shopping_center_id'].unique())})
result.head()

,shopping_center_index,shopping_center_id
0,0,b43e9e4f-acd1-4941-874d-e0c5650ab91e
1,1,599cb959-11ef-49aa-9eb3-e6c17b4ea6ba
2,2,0cd35523-1eca-4f09-ab0d-0b506ae9d986
3,3,cb2d5bb6-c372-4a51-8231-4ffa288a0c28


Fonction qui permet d'arrondir l'heure aux 5 minutes les plus proches.

In [127]:
def round_nearest(x, a):
    result =  round(x / a) * a
    if str(result).find('.6') == 1:
        result += 0.4
    return result

Ma solution est de determiner les horaires d'ouvertures et de fermetures en gardant uniquement les horaires qui representent 90% de l'affluences. J'ai donc exclut les 5 premiers et les 5 derniers percentiles. 

Pendant le calcul de mon resultat, j'exclut egalement les multiples pings d'une meme personne pendant la meme heure afin de ne pas biaiser les resultats.

## Le tableau ci dessous presente les resultats de l'algorithme.

In [128]:
weekDays = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
for i in range(7):
  df_shop = df.loc[df['day'] == i]
  openhours = []
  closehours = []
  for j in range(len(df_shop.shopping_center_index.unique())):
    data = df_shop.loc[df_shop['shopping_center_index'] == j]
    data['numeric'] = data['device_local_date'].dt.hour + data['device_local_date'].dt.minute/60.0
    data = data.sort_values(by=['numeric'])
    data = data.drop_duplicates(subset=['device_hash_id'])
    tmp = round(float(data.numeric.quantile([0.05])), 2)
    openhours.append(round_nearest(float('{0:02.0f}.{1:02.0f}'.format(*divmod(tmp * 60, 60))), 0.05))
    tmp = round(float(data.numeric.quantile([0.95])), 2)
    closehours.append(round_nearest(float('{0:02.0f}.{1:02.0f}'.format(*divmod(tmp * 60, 60))), 0.05))
  result[weekDays[i] + '_open'] = openhours
  result[weekDays[i] + '_close'] = closehours
result.loc[result['shopping_center_index'] == 0]
result = result.drop(columns='shopping_center_index')
result.head(len(df_shop.shopping_center_index.unique()))

,shopping_center_id,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,Thursday_close,Friday_open,Friday_close,Saturday_open,Saturday_close,Sunday_open,Sunday_close
0,b43e9e4f-acd1-4941-874d-e0c5650ab91e,8.0,19.35,8.00,19.30,8.25,19.50,8.40,19.20,8.35,20.20,8.20,19.25,5.40,20.20
1,599cb959-11ef-49aa-9eb3-e6c17b4ea6ba,8.3,19.45,8.15,19.45,8.40,19.35,8.20,19.45,8.45,20.15,9.45,19.35,6.35,20.25
2,0cd35523-1eca-4f09-ab0d-0b506ae9d986,9.1,19.55,9.05,19.40,9.05,19.45,8.25,19.15,9.00,20.00,9.25,19.35,7.45,20.40
3,cb2d5bb6-c372-4a51-8231-4ffa288a0c28,8.3,19.45,7.40,19.55,8.25,20.10,9.00,20.15,8.45,21.35,9.15,20.20,9.25,19.10


# Pour aller plus loin

Je pense qu'il serait intéressant d'avoir le cout d'opération. En effet toutes les dépenses sont impactées lorsque le magasin est ouvert plus longtemps. Il serait également pertinent d'obtenir le ticket moyen d'un client. C'est 2 données supplémentaires associées à la frequentation des clients permettraient de calculer un seuil de rentabilité qui nous permettrait de choisir avec d'avantages de pertinences les horaires d'ouvertures et de fermetures d'un magasin.